In [ ]:
# !pip3 install osmnx
!pip3 install geojson
!pip3 install shapely.constructive
!pip install geopandas
# !sudo apt install libspatialindex-dev
# !pip install rtree
# !pip3 uninstall numpy
# !pip3 install numpy
# !pip uninstall rtree
# import rtree
import pandas as pd
import numpy as np
# import csv
import plotly.express as px
!pip install osmpy
import osmpy
import geopandas as gpd
import json
import requests
%matplotlib inline
from shapely.geometry import Point
from geopandas import datasets, GeoDataFrame, read_file
!wget https://raw.githubusercontent.com/NHERI-SimCenter/BRAILS/master/brails/workflow/FootprintHandler.py
import FootprintHandler
from FootprintHandler import FootprintHandler
from google.colab import drive

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached geojson-3.0.1-py3-none-any.whl (15 kB)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement shapely.constructive (from versions: none)
ERROR: No matching distribution found for shapely.constructive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached osmpy-0.2.0-py3-none-any.whl (5.2 kB)
  Using cached Shapely-1.7.0-cp38-cp38-manylinux1_x86_64.whl (1.8 MB)
  Using cached retry-0.9.2-py2.py3-none-any.whl (8.0 kB)
  Using cached geojson-2.5.0-py2.py3-none-any.whl (14 kB)
  Using cached requests-2.23.0-py2.py3-none-any.whl (58 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 1.8 MB/s eta 0:00:00
  U

/usr/local/lib/python3.8/site-packages/pyproj/__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
workdir = '/content/drive/MyDrive/Colab Notebooks/Hazus Data/' # for Tina
# workdir = '/content/drive/MyDrive/2022_Tina_Hazus_Database/'                 # for Adam

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Loading Files and Data Cleaning Section

We first input the state abbrivation, FIPS code, and county of the area of interest

In [ ]:
state = 'CA'
fips_code = '06001'
county_name = 'Alameda_County'

In [ ]:
path = f'{workdir}Processed_Data/{state}/{state}_{fips_code}_{county_name}.csv'
example = pd.read_csv(path, low_memory = False, index_col = 0, header = [0,1])

In [ ]:
area = 'Emeryville, California'

In [ ]:
fp_handler = FootprintHandler()
fp_handler.fetch_footprint_data(area)
footprints = gpd.GeoDataFrame(geometry=[Polygon(fp) for fp in fp_handler.footprints])
footprints.head()


Searching for Emeryville, California...
Found Emeryville, Alameda County, California, 94608, United States

Fetching footprint data for Emeryville...
Found a total of 1207 building footprints in Emeryville


,geometry
0,"POLYGON ((-122.28244 37.82890, -122.28242 37.8..."
1,"POLYGON ((-122.28744 37.82934, -122.28729 37.8..."
2,"POLYGON ((-122.28605 37.82959, -122.28604 37.8..."
3,"POLYGON ((-122.29244 37.83239, -122.29244 37.8..."
4,"POLYGON ((-122.29143 37.83758, -122.29077 37.8..."


In [ ]:
# fig = px.choropleth_mapbox(
#     footprints,
#     geojson=footprints["geometry"],
#     locations=footprints.index,
#     center={"lat": 37.8384729, "lon": -122.3016820},
#     mapbox_style="carto-positron",
#     color_continuous_scale='Portland',
#     opacity=0.9,
#     zoom=13,
#     height=800,
#     width=1000,
#     title = "Emeryville Footprints"
#   )
# fig.show()

In [ ]:
# # compute city of interest here
# place_name = 'Emeryville, California'
# footprints = ox.geometries_from_place(place_name, tags = {'building': True})
# footprints.index = np.arange(footprints.shape[0])

In [ ]:
example = example[[('Demographics'),('BuildingCount'),('TIGERCensusBlock')]]
example[('Demographics'),"BuiltPre1950"] = example[('Demographics'),"Built40to49"] +example[('Demographics'),"BuiltBefore40"]
example[('Demographics'),"Built50to70"] = example[('Demographics'),"Built50to59"] +example[('Demographics'),"Built60to69"]
example[('Demographics'),"BuiltPost70"] = example[('Demographics'),"Built70to79"] +example[('Demographics'),"Built80to89"] +example[('Demographics'),"Built90to98"]+example[('Demographics'),"BuiltAfter98"]
buildingcount = example[('BuildingCount')]

<ipython-input-51-e135877a2449>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-51-e135877a2449>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-51-e135877a2449>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



We then filter out all of the usesful informations from BuildingCount, Demographics, and TIGERCensusBlock table and conbine then togather.

In [ ]:
rest_count = pd.concat([buildingcount["RES2"],buildingcount.iloc[:,8:33]], axis = 1)
buildingcount["sum"] = buildingcount.sum(axis = 1)
buildingcount["rest_sum"] = rest_count.sum(axis = 1)
buildingcount["RES3_sum"] = buildingcount.iloc[:,2:8].sum(axis = 1)
col = ['RES1', 'RES3A', 'RES3B', 'RES3C', 'RES3D', 'RES3E', 'RES3F','RES2',
       'RES4', 'RES5', 'RES6', 'COM1', 'COM2', 'COM3', 'COM4', 'COM5', 'COM6',
       'COM7', 'COM8', 'COM9', 'COM10', 'IND1', 'IND2', 'IND3', 'IND4', 'IND5',
       'IND6', 'AGR1', 'REL1', 'GOV1', 'GOV2', 'EDU1', 'EDU2', 'RES3_sum',
       'rest_sum', 'sum']
buildingcount = buildingcount[col]
buildingcount.columns = buildingcount.columns.to_flat_index()

example.columns = example.columns.to_flat_index()
year = example[[('Demographics', 'BuiltPre1950'),
                ('Demographics', 'Built50to70'),
                ('Demographics', 'BuiltPost70')]]
year = year.rename(columns = {('Demographics', 'BuiltPre1950') : 'BuiltPre1950',
                ('Demographics', 'Built50to70'):'Built50to70',
                ('Demographics', 'BuiltPost70'):'BuiltPost70'})

rise = example[[('TIGERCensusBlock', 'Pct1StoryRes1'),('TIGERCensusBlock', 'Pct2StoryRes1'),
                ('TIGERCensusBlock', 'Pct3StoryRes1'),('TIGERCensusBlock', 'PctSplitLvlRes1'),
                ('TIGERCensusBlock', 'Pct1to2StryRes3'),
                ('TIGERCensusBlock', 'Pct3to4StryRes3'),('TIGERCensusBlock', 'Pct5StryplusRes3'),
                ('TIGERCensusBlock', 'PctHighRiseOther'),
                ('TIGERCensusBlock', 'PctLowRiseOther'),('TIGERCensusBlock', 'PctMidRiseOther')]]
rise = rise.rename(columns={('TIGERCensusBlock', 'Pct1StoryRes1'):"RES1,1",
          ('TIGERCensusBlock', 'Pct2StoryRes1'):"RES1,2",
          ('TIGERCensusBlock', 'Pct3StoryRes1'):"RES1,3",
        ('TIGERCensusBlock', 'PctSplitLvlRes1'):"RES1,1.5",
        ('TIGERCensusBlock', 'Pct1to2StryRes3'):"RES3,1-2",
        ('TIGERCensusBlock', 'Pct3to4StryRes3'):"RES3,3-4",
       ('TIGERCensusBlock', 'Pct5StryplusRes3'):"RES3,5+",
       ('TIGERCensusBlock', 'PctHighRiseOther'):"HighRise,8+",
        ('TIGERCensusBlock', 'PctLowRiseOther'):"LowRise,1-3",
        ('TIGERCensusBlock', 'PctMidRiseOther'):"MidRise,4-7"})
rise["RES1,1"] = rise["RES1,1"] + rise["RES1,1.5"]
rise["RES3,3"] = rise["RES3,3-4"]/2
rise["RES3,4"] = rise["RES3,3-4"]/2
rise = rise.drop(["RES1,1.5","RES3,3-4"],axis = 1)
rise = rise[['RES1,1', 'RES1,2', 'RES1,3', 'RES3,1-2', 'RES3,3', 'RES3,4','RES3,5+','HighRise,8+', 'LowRise,1-3', 'MidRise,4-7']]
sum_table = pd.concat([buildingcount, year,rise], axis = 1)
sum_table.head()

<ipython-input-52-fa6438a74d40>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-52-fa6438a74d40>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-52-fa6438a74d40>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,RES1,RES3A,RES3B,RES3C,RES3D,RES3E,RES3F,RES2,RES4,RES5,...,"RES1,1","RES1,2","RES1,3","RES3,1-2","RES3,3","RES3,4","RES3,5+","HighRise,8+","LowRise,1-3","MidRise,4-7"
CensusBlock,,,,,,,,,,,,,,,,,,,,,
60014001001000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.68,0.3,0.02,0.73,0.11,0.11,0.05,0.01,0.98,0.01
60014001001001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.68,0.3,0.02,0.73,0.11,0.11,0.05,0.01,0.98,0.01
60014001001002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.68,0.3,0.02,0.73,0.11,0.11,0.05,0.01,0.98,0.01
60014001001003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.68,0.3,0.02,0.73,0.11,0.11,0.05,0.01,0.98,0.01
60014001001004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.68,0.3,0.02,0.73,0.11,0.11,0.05,0.01,0.98,0.01


In [ ]:
!pip install --upgrade pyproj

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


This is polygon file which gives the latitude and longtitude information

In [ ]:
CT_poly = gpd.read_file(f'{workdir}Processed_Data/{state}/tl_2010_{fips_code[:2]}_tabblock10.zip')

PROJ: proj_create_from_database: /usr/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 0 whereas a number >= 2 is expected. It comes from another PROJ installation.


In [ ]:
CT_poly

,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE10,GEOID10,NAME10,MTFCC10,UR10,UACE10,UATYP10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry
0,06,001,410000,2024,060014100002024,Block 2024,G5040,U,78904,U,S,1180,0,+37.7514499,-122.1244488,"POLYGON ((-122.12489 37.75164, -122.12476 37.7..."
1,06,001,407300,2017,060014073002017,Block 2017,G5040,U,78904,U,S,553,0,+37.7700438,-122.2099801,"POLYGON ((-122.21025 37.77010, -122.21014 37.7..."
2,06,001,408100,2011,060014081002011,Block 2011,G5040,U,78904,U,S,11797,0,+37.7945823,-122.1820803,"POLYGON ((-122.18212 37.79493, -122.18184 37.7..."
3,06,001,401000,2002,060014010002002,Block 2002,G5040,U,78904,U,S,598,0,+37.8343484,-122.2699940,"POLYGON ((-122.27004 37.83399, -122.27013 37.8..."
4,06,001,406100,1005,060014061001005,Block 1005,G5040,U,78904,U,S,139,0,+37.7783082,-122.2273093,"POLYGON ((-122.22720 37.77830, -122.22726 37.7..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
710140,06,115,041100,4004,061150411004004,Block 4004,G5040,R,None,None,S,583474,0,+39.4903865,-121.2958678,"POLYGON ((-121.30099 39.48986, -121.30102 39.4..."
710141,06,115,041100,3015,061150411003015,Block 3015,G5040,R,None,None,S,31015,0,+39.4554251,-121.2687177,"POLYGON ((-121.26819 39.45691, -121.26794 39.4..."
710142,06,115,041100,2032,061150411002032,Block 2032,G5040,R,None,None,S,80959,0,+39.3781732,-121.2645753,"POLYGON ((-121.26237 39.37909, -121.26248 39.3..."
710143,06,115,041100,2063,061150411002063,Block 2063,G5040,R,None,None,S,31921,0,+39.3765805,-121.2710594,"POLYGON ((-121.26991 39.37525, -121.27062 39.3..."


In [ ]:
position = CT_poly.sjoin(footprints,how = "inner")[["GEOID10","geometry"]]
position["GEOID10"] = position["GEOID10"].astype(str).astype(int)
position.head()

/usr/local/lib/python3.8/site-packages/geopandas/geodataframe.py:2061: UserWarning:

CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: GEOGCS["GCS_North_American_1983",DATUM["D_North_Am ...
Right CRS: None




,GEOID10,geometry
267,60014251032015,"POLYGON ((-122.28920 37.84528, -122.28954 37.8..."
267,60014251032015,"POLYGON ((-122.28920 37.84528, -122.28954 37.8..."
267,60014251032015,"POLYGON ((-122.28920 37.84528, -122.28954 37.8..."
267,60014251032015,"POLYGON ((-122.28920 37.84528, -122.28954 37.8..."
273,60014251031017,"POLYGON ((-122.29126 37.83490, -122.29135 37.8..."


In [ ]:
# fig = px.choropleth_mapbox(
#     position,
#     geojson=position.geometry,
#     locations=position.index,
#     center={"lat": 37.8384729, "lon": -122.3016820},
#     mapbox_style="carto-positron",
#     color_continuous_scale='Portland',
#     opacity=0.9,
#     zoom=13,
#     height=800,
#     width=1000,
#     title = "Emeryville Footprints"
#   )
# fig.show(renderer="colab")

# Simulation Section

In [ ]:
# We merge the polygon with the previous table and set the censusblock id as the index
em = sum_table.merge(position, left_index= True, right_on = "GEOID10")
em = em.set_index("GEOID10")

# Since the building year columns are in percentage we convert them to acutal building counts within each censusblock
em["BuiltPre1950_percent"] = np.around((em["BuiltPre1950"] / (em["BuiltPre1950"] + em["Built50to70"] + em["BuiltPost70"])) *em["sum"].astype(int)).fillna(0)
em["Built50to70_percent"] = np.around((em["Built50to70"] / (em["BuiltPre1950"] + em["Built50to70"] + em["BuiltPost70"])) *em["sum"].astype(int)).fillna(0)
em["BuiltPost70_percent"] = np.around((em["BuiltPost70"] / (em["BuiltPre1950"] + em["Built50to70"] + em["BuiltPost70"])) *em["sum"].astype(int)).fillna(0)
em["BuiltPre1950"] = em["BuiltPre1950_percent"]
em["Built50to70"] = em["Built50to70_percent"]
em["BuiltPost70"] = em["BuiltPost70_percent"]

# Res1 and Res3 are different from other types, we did some convertions for RES1 buildings
em["RES1,1"] = np.around(em['RES1,1']*em["RES1"].astype(int).fillna(0))
em["RES1,2"] = np.around(em['RES1,2']*em["RES1"].astype(int).fillna(0))
em["RES1,3"] = np.around(em['RES1,3']*em["RES1"].astype(int).fillna(0))
em["RES3,1-2"] = np.around(em['RES3,1-2']*em["RES3_sum"].astype(int).fillna(0))
em["RES3,3"] = np.around(em['RES3,3']*em["RES3_sum"].astype(int).fillna(0))
em["RES3,4"] = np.around(em['RES3,4']*em["RES3_sum"].astype(int).fillna(0))
em["RES3,5+"] = em["RES3_sum"] - em["RES3,1-2"] - em["RES3,3"] - em["RES3,4"]

# Since the building rise columns are in percentage we convert them to acutal bulding counts within each censusblock
em["HighRise,8+"] = np.around(em['HighRise,8+']*em["rest_sum"].astype(int).fillna(0))
em["LowRise,1-3"] = np.around(em["LowRise,1-3"]) *em["rest_sum"].astype(int).fillna(0)
em["MidRise,4-7"] = em["rest_sum"] - em["HighRise,8+"] - em["LowRise,1-3"]
em = em.drop([("BuiltPre1950_percent"),("Built50to70_percent"),("BuiltPost70_percent")], axis = 1)
geo_gpd = gpd.GeoDataFrame(em)
geo_gpd["Lon"] = geo_gpd["geometry"].centroid.x
geo_gpd["Lat"] = geo_gpd["geometry"].centroid.y
geo_gpd = geo_gpd.drop("geometry",axis = 1)
em = geo_gpd
em

<ipython-input-65-bb4999559329>:28: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


<ipython-input-65-bb4999559329>:29: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




,RES1,RES3A,RES3B,RES3C,RES3D,RES3E,RES3F,RES2,RES4,RES5,...,"RES1,3","RES3,1-2","RES3,3","RES3,4","RES3,5+","HighRise,8+","LowRise,1-3","MidRise,4-7",Lon,Lat
GEOID10,,,,,,,,,,,,,,,,,,,,,
60014008001005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,-122.287479,37.850046
60014010006004,52.0,0.0,5.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,7.0,1.0,1.0,0.0,0.0,1.0,0.0,-122.276279,37.829492
60014015001000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-122.279089,37.827153
60014017001001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-122.284081,37.828079
60014017001004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-122.286619,37.828783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60014251042050,6.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-122.285053,37.836150
60014251042050,6.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-122.285053,37.836150
60014251042050,6.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-122.285053,37.836150


This is the function that expands all buildings within censusblock. From now on, each row represent one building.

In [ ]:
def convertTocount(r):
  # we create five array conresponding with buliding type, building year, building rise, lat and lon.
  # we also shuffled the buildings to ensure the independence of each columns
  building = np.arange(0,em.iloc[r]["sum"].astype(int))
  np.random.shuffle(building)
  #
  occarr = []
  for i in range(0,33):
    occarr += [em.iloc[r].index[i],] * em.iloc[r][em.columns[i]].astype(int)
  #
  yeararr = []
  for i in range(36,39):
    yeararr += [em.iloc[r].index[i],] * em.iloc[r][em.columns[i]].astype(int)
  if len(yeararr) == 0:
    yeararr = [em.iloc[r].index[38],] * em.iloc[r]["sum"].astype(int)
  if len(yeararr) < em.iloc[r][em.columns[35]]:
    yeararr += [em.iloc[r].index[36],]
  if len(yeararr) > em.iloc[r][em.columns[35]]:
    yeararr = yeararr[1:]
  #
  risearr = []
  for i in range(39,49):
      risearr += [em.iloc[r].index[i],] * em.iloc[r][em.columns[i]].astype(int)
  if len(risearr) > em.iloc[r][em.columns[35]]:
    risearr = risearr[1:]
  if len(risearr) < em.iloc[r][em.columns[35]]:
    risearr += [em.iloc[r].index[39],]
  Lat = [em.iloc[r,49].astype(str)] * em.iloc[r]["sum"].astype(int)
  Lon = [em.iloc[r,50].astype(str)] * em.iloc[r]["sum"].astype(int)
  subrow = pd.DataFrame(
        {
            "building":building,
            "Censusblock":em.index[r],
            "OccType":occarr,
            "Year":yeararr,
            "Rise":risearr,
            "Lat":Lat,
            "Lon":Lon
        }
    )
  return subrow

In [ ]:
# We created an empty table to start with
subrow = pd.DataFrame(
      {
          "building":[],
          "Censusblock":[],
          "OccType":[],
          "Year":[],
          "Rise":[]
      }
    )

We assigned all  buildings with their type, year,rise, lat, and lon within the area of interests

In [ ]:
for i in range(len(em.index)):
  try:
    tbl = convertTocount(i)
    subrow = pd.concat([subrow,tbl])
  except ValueError:
    print(i)
subrow["building"] = subrow["building"].astype(int)
subrow["Censusblock"] = subrow["Censusblock"].astype(int)
subrow.reset_index(inplace = True)

In [ ]:
buildings = subrow.copy()
buildings

,index,building,Censusblock,OccType,Year,Rise,Lat,Lon
0,0,0,60014008001005,COM2,BuiltPost70,"LowRise,1-3",-122.28747943229963,37.85004566417667
1,1,1,60014008001005,EDU2,BuiltPost70,"LowRise,1-3",-122.28747943229963,37.85004566417667
2,0,60,60014010006004,RES1,BuiltPre1950,"RES1,1",-122.27627927055016,37.82949151837393
3,1,48,60014010006004,RES1,BuiltPre1950,"RES1,1",-122.27627927055016,37.82949151837393
4,2,41,60014010006004,RES1,BuiltPre1950,"RES1,1",-122.27627927055016,37.82949151837393
...,...,...,...,...,...,...,...,...
34583,2,4,60014251042050,RES1,Built50to70,"RES1,1",-122.28505279206823,37.836149678075685
34584,3,1,60014251042050,RES1,BuiltPost70,"RES1,1",-122.28505279206823,37.836149678075685
34585,4,2,60014251042050,RES1,BuiltPost70,"RES1,2",-122.28505279206823,37.836149678075685
34586,5,3,60014251042050,RES1,BuiltPost70,"RES1,2",-122.28505279206823,37.836149678075685


# Sampling Section

In this section, we will assign building material based on the probability table from Hazus. In our modeling, Rise 1-3 belongs t0 Low-Rise, 4-7 belongs to Mid-Rise and 8+ belongs to High-Rise.

In [ ]:
buildings = buildings.drop(["index"],axis = 1)
buildings["Rise"] = buildings["Rise"].str.split(pat=",").str[1]
buildings["Year"] = buildings["Year"].replace( {"BuiltPre1950": "Pre-1950", "Built50to70":"1950-1970","BuiltPost70":"Post-1970"})
buildings["Type"] = ""
buildings ["Rise_Before"] = buildings ["Rise"]
buildings ["Rise"] = buildings["Rise"].replace({"1":"Low-Rise","1-2":"Low-Rise","3":"Low-Rise", "4":"Mid-Rise","5+":"Mid-Rise","1-3":"Low-Rise", "4-7":"Mid-Rise","8+":"High-Rise"})
buildings.sort_values("Rise")

,building,Censusblock,OccType,Year,Rise,Lat,Lon,Type,Rise_Before
10643,6,60014251033007,RES1,Post-1970,2,-122.28644534874695,37.84103432673043,,2
8355,7,60014251033004,RES1,1950-1970,2,-122.28765418769204,37.84352665117224,,2
23666,47,60014251041002,RES1,Pre-1950,2,-122.27870023011124,37.83648082684696,,2
23667,31,60014251041002,RES1,Pre-1950,2,-122.27870023011124,37.83648082684696,,2
23668,29,60014251041002,RES1,Pre-1950,2,-122.27870023011124,37.83648082684696,,2
...,...,...,...,...,...,...,...,...,...
15593,50,60014251041001,RES3B,Post-1970,Mid-Rise,-122.27882270804416,37.835605747533656,,4
15594,42,60014251041001,RES3C,Post-1970,Mid-Rise,-122.27882270804416,37.835605747533656,,4
15595,47,60014251041001,RES3D,Post-1970,Mid-Rise,-122.27882270804416,37.835605747533656,,5+
7423,4,60014251033003,RES3B,Post-1970,Mid-Rise,-122.28793088227211,37.84424467774214,,4


We load the coast table here. The coast table contains the information about state and coast.

In [ ]:
coast = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Hazus Data/Regional Distribution of States .csv")
coastname = coast.loc[coast["State Abbreviation"] == state]["Group"].values[0]

In [ ]:
# filter out coast name here
coastname

'West Coast'

Once we know the coast name, year, rise, and building type, we should know to filter out the coresponeded probability table to generate the building material for each building.

In [ ]:
# filter out tables related to west coast
def westfilter(file_name,buildings):
  file_name_arr = file_name.replace(", ",",")
  file_name_arr = file_name_arr.split(",")
  tbl = buildings.loc[(buildings["Year"] == file_name_arr[1])]
  tbl = tbl.loc[(tbl["Rise"] == file_name_arr[0]) & (buildings["OccType"] != "RES1")]
  return tbl

In [ ]:
# function used to generate RES1
from scipy.stats import multinomial
def res1_west_generator(file_name, btype_tbl, buildings, occup_class):
  file_name_arr = file_name.replace(", ",",")
  file_name_arr = file_name_arr.split(",")
  if file_name_arr[0] == "RES1":
    filtered_bldgs = buildings.loc[(buildings["OccType"] == file_name_arr[0]) & (buildings["Year"] == file_name_arr[1])]
  length = len(filtered_bldgs)
  rng = np.random.default_rng()
  state_index = btype_tbl.loc[btype_tbl["State\nAbbreviation"] == state].index
  state_index = state_index.values[0]
  rv = rng.multinomial(length,[(btype_tbl.iloc[state_index,3]/100),(btype_tbl.iloc[state_index,4]/100),(btype_tbl.iloc[state_index,5]/100),(btype_tbl.iloc[state_index,6]/100),(btype_tbl.iloc[state_index,7]/100),(btype_tbl.iloc[state_index,8]/100)], size = 1)[0]
  bldg_type = []
  type_arr = btype_tbl.columns[3:]
  for i in range(len(rv)):
    bldg_type += rv[i] * [type_arr[i]]
  np.random.shuffle(bldg_type)
  filtered_bldgs["Type"] = bldg_type
  return filtered_bldgs

In [ ]:
from scipy.stats import multinomial

def west_generator(file_name,buildings):
  filtered_bldgs = westfilter(file_name,buildings)
    # loop through each row within the table Res2, Res3...
  btype_tbl = pd.read_csv(f'/content/drive/MyDrive/Colab Notebooks/Hazus Data/{file_name}.csv',header = 1)
  btype_tbl = btype_tbl.fillna(0)
  # get all buildings types from the probability table and get their building material distribution
  for r in range(len(btype_tbl)):
    occname = btype_tbl.iloc[r,0]
    occup_tbl = filtered_bldgs.loc[filtered_bldgs["OccType"].str.contains(occname)]
    length = len(occup_tbl)
    rng = np.random.default_rng()
    dic = btype_tbl.iloc[r,1:].to_dict()
    dic_value = list(dic.values())
    dic_key = []
    for k in range(len(list(dic.values()))):
      dic_value[k] = dic_value[k]/100
      dic_key.append(list(dic.keys())[k])
    rv = rng.multinomial(length,dic_value, size = 1)[0]
    bldg_type = []
    # put the sampled building materials back to buildings table
    for j in range(len(rv)):
      bldg_type += rv[j] * [dic_key[j]]
    np.random.shuffle(bldg_type)
    filtered_bldgs.loc[filtered_bldgs["OccType"].str.contains(occname),"Type"] = bldg_type
  return filtered_bldgs

Mid-West and East Coast functions:

In [ ]:
# function used to generate RES1
def res1_MidwestAndEast_generator(file_name, btype_tbl, buildings, occup_class):
  file_name_arr = file_name.replace(", ",",")
  file_name_arr = file_name_arr.split(",")
  if file_name_arr[0] == "RES1":
    filtered_bldgs = buildings.loc[(buildings["OccType"] == file_name_arr[0])]
  length = len(filtered_bldgs)
  rng = np.random.default_rng()
  state_index = btype_tbl.loc[btype_tbl["State\nAbbreviation"] == state].index
  state_index = state_index.values[0]
  dic = btype_tbl.iloc[state_index,3:].to_dict()
  dic_value = list(dic.values())
  dic_key = []
  for k in range(len(list(dic.values()))):
    dic_value[k] = dic_value[k]/100
    dic_key.append(list(dic.keys())[k])
  rv = rng.multinomial(length,dic_value, size = 1)[0]
  bldg_type = []
  for j in range(len(rv)):
    bldg_type += rv[j] * [dic_key[j]]
  np.random.shuffle(bldg_type)
  filtered_bldgs["Type"] = bldg_type
  return filtered_bldgs

In [ ]:
# filter out tables related to east coast and mid-west
def otherfilter(file_name,buildings):
  file_name_arr = file_name.replace(", ",",")
  file_name_arr = file_name_arr.split(",")
  tbl = buildings.loc[(buildings["Rise"] == file_name_arr[0]) & (buildings["OccType"] != "RES1")]
  return tbl

In [ ]:
from scipy.stats import multinomial

def MidEast_generator(file_name,buildings):
  filtered_bldgs = otherfilter(file_name,buildings)
  # tables contains probabilities from each type
  btype_tbl = pd.read_csv(f'/content/drive/MyDrive/Colab Notebooks/Hazus Data/{file_name}.csv',header = 1)
  btype_tbl = btype_tbl.fillna(0)
  # same as west_generator but only filter out rises since mid-west and east doesn't uses years
  for r in range(len(btype_tbl)):
    occname = btype_tbl.iloc[r,0]
    occup_tbl = filtered_bldgs.loc[filtered_bldgs["OccType"].str.contains(occname)]
    length = len(occup_tbl)
    rng = np.random.default_rng()
    dic = btype_tbl.iloc[r,1:].to_dict()
    dic_value = list(dic.values())
    for c in range(len(list(dic.values()))):
      dic_value[c] = int(dic_value[c])
    dic_key = []
    for k in range(len(list(dic.values()))):
      dic_value[k] = dic_value[k]/100
      dic_key.append(list(dic.keys())[k])
    rv = rng.multinomial(length,dic_value, size = 1)[0]
    bldg_type = []
    for j in range(len(rv)):
      bldg_type += rv[j] * [dic_key[j]]
    np.random.shuffle(bldg_type)
    filtered_bldgs.loc[filtered_bldgs["OccType"].str.contains(occname),"Type"] = bldg_type
  return filtered_bldgs

In [ ]:
def sample_buildingtype(coastname):
  building_type = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Hazus Data/Building_Type.csv")
  building_type = building_type.sort_values(["RES1","Coast"])
  building_type = building_type.loc[building_type["Coast"] == coastname]
  building_type = building_type.fillna("")
  # West coast table
  if coastname == "West Coast":
    #Generate RES1 Table
    res1_tbl = pd.DataFrame()
    for i in range(3):
      file_name= f'{building_type.iloc[i,0]}{building_type.iloc[i,1]}{building_type.iloc[i,2]}{building_type.iloc[i,3]}'
      btype_tbl = pd.read_csv(f'/content/drive/MyDrive/Colab Notebooks/Hazus Data/{file_name}.csv',header = 1)
      btype_tbl = btype_tbl.fillna(0)
      test_table = res1_west_generator(file_name, btype_tbl, buildings, "RES1")
      res1_tbl = pd.concat([res1_tbl,test_table],axis = 0)
    # sample other occtypes except RES1
    table = pd.DataFrame()
    for i in range(3,len(building_type)):
      file_name= f'{building_type.iloc[i,0]}{building_type.iloc[i,1]}{building_type.iloc[i,2]}{building_type.iloc[i,3]}'
      test_table = west_generator(file_name,buildings)
      table = pd.concat([table,test_table],axis = 0)
    sum_table = pd.concat([res1_tbl,table])
  # Mid-West and East coast table
  else:
    file_name= f'{building_type.iloc[0,0]}{building_type.iloc[0,1]}{building_type.iloc[0,2]}{building_type.iloc[0,3]}'
    btype_tbl = pd.read_csv(f'/content/drive/MyDrive/Colab Notebooks/Hazus Data/{file_name}.csv',header = 1)
    btype_tbl = btype_tbl.fillna(0)
    res1_tbl = res1_MidwestAndEast_generator(file_name, btype_tbl, buildings, "RES1")

    table = pd.DataFrame()
    for i in range(1,4):
      file_name= f'{building_type.iloc[i,0]}{building_type.iloc[i,1]}{building_type.iloc[i,2]}{building_type.iloc[i,3]}'
      test_table = MidEast_generator(file_name,buildings)
      table = pd.concat([table,test_table],axis = 0)
    sum_table = pd.concat([res1_tbl,table])
  return sum_table

By using sample_buildingtype function, we will generate the building materials based on the probability table given by Hazus

In [ ]:
sample_buildingtype(coastname)

<ipython-input-75-12cbebe2858e>:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-75-12cbebe2858e>:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-75-12cbebe2858e>:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,building,Censusblock,OccType,Year,Rise,Lat,Lon,Type,Rise_Before
2,60,60014010006004,RES1,Pre-1950,Low-Rise,-122.27627927055016,37.82949151837393,W1,1
3,48,60014010006004,RES1,Pre-1950,Low-Rise,-122.27627927055016,37.82949151837393,W1,1
4,41,60014010006004,RES1,Pre-1950,Low-Rise,-122.27627927055016,37.82949151837393,W1,1
5,42,60014010006004,RES1,Pre-1950,Low-Rise,-122.27627927055016,37.82949151837393,W1,1
6,61,60014010006004,RES1,Pre-1950,Low-Rise,-122.27627927055016,37.82949151837393,W1,1
...,...,...,...,...,...,...,...,...,...
34559,5,60014251042050,RES3B,Post-1970,Low-Rise,-122.28505279206823,37.836149678075685,W1,1-2
34566,2,60014251042050,RES3B,Post-1970,Low-Rise,-122.28505279206823,37.836149678075685,W1,1-2
34573,0,60014251042050,RES3B,Post-1970,Low-Rise,-122.28505279206823,37.836149678075685,MH,1-2
34580,2,60014251042050,RES3B,Post-1970,Low-Rise,-122.28505279206823,37.836149678075685,W1,1-2
